In [1]:
from methods import LBFGS, Newton, BFGS, LBFGS, LineSearchTool
import numpy as np
import scipy

In [2]:
class BaseSmoothOracle(object):
    """
    Base class for implementation of oracles.
    """
    def func(self, x):
        """
        Computes the value of function at point x.
        """
        raise NotImplementedError('Func oracle is not implemented.')

    def grad(self, x):
        """
        Computes the gradient at point x.
        """
        raise NotImplementedError('Grad oracle is not implemented.')
    
    def func_directional(self, x, d, alpha):
        """
        Computes phi(alpha) = f(x + alpha*d).
        """
        return np.squeeze(self.func(x + alpha * d))

    def grad_directional(self, x, d, alpha):
        """
        Computes phi'(alpha) = (f(x + alpha*d))'_{alpha}
        """
        return np.squeeze(self.grad(x + alpha * d).dot(d))


In [3]:
class QuadraticOracle(BaseSmoothOracle):
    def __init__(self, A, b):
        if not scipy.sparse.isspmatrix_dia(A) and not np.allclose(A, A.T):
            raise ValueError('A should be a symmetric matrix.')
        self.A = A
        self.b = b

    def func(self, x):
        return 0.5 * np.dot(self.A.dot(x), x) - self.b.dot(x)

    def grad(self, x):
        return self.A.dot(x) - self.b

    def hess(self, x):
        return self.A


In [4]:
class TestNewton():
    def get_quadratic(self):
        # Quadratic function:
        #   f(x) = 1/2 x^T x - [1, 2, 3]^T x
        A = np.eye(3)
        b = np.array([1, 2, 3])
        return oracles.QuadraticOracle(A, b)


    def test_newton_ideal_step(self):
        oracle = self.get_quadratic()
        x0 = np.ones(3) * 10.0
        opt = Newton(oracle, x0, tolerance=1e-5)
        opt.run(10)
        ok_(np.allclose(opt.hist['x_star'], [1.0, 2.0, 3.0]))
        check_equal_histories(opt.hist, {'func': [90.0, -7.0],
                                        'grad_norm': [13.928388277184119, 0.0],
                                        'time': [0, 1]  # dummy timestamps
                                        })


    def get_1d(self, alpha):
        # 1D function:
        #   f(x) = exp(alpha * x) + alpha * x^2
        class Func(oracles.BaseSmoothOracle):
            def __init__(self, alpha):
                self.alpha = alpha

            def func(self, x):
                return np.exp(self.alpha * x) + self.alpha * x ** 2

            def grad(self, x):
                return np.array(self.alpha * np.exp(self.alpha * x) +
                                2 * self.alpha * x)

            def hess(self, x):
                return np.array([self.alpha ** 2 * np.exp(self.alpha * x) +
                                 2 * self.alpha])

        return Func(alpha)

In [4]:
    def get_1d(alpha):
        # 1D function:
        #   f(x) = exp(alpha * x) + alpha * x^2
        class Func(BaseSmoothOracle):
            def __init__(self, alpha):
                self.alpha = alpha

            def func(self, x):
                return np.exp(self.alpha * x) + self.alpha * x ** 2

            def grad(self, x):
                return np.array(self.alpha * np.exp(self.alpha * x) +
                                2 * self.alpha * x)

            def hess(self, x):
                return np.array([self.alpha ** 2 * np.exp(self.alpha * x) +
                                 2 * self.alpha])

        return Func(alpha)

In [5]:
oracle = get_1d(0.5)
x0 = np.array([1.0])
FUNC = [
    np.array([2.14872127]),
    np.array([0.9068072]),
    np.array([0.89869455]),
    np.array([0.89869434])]
GRAD_NORM = [
    1.8243606353500641,
    0.14023069594489929,
    0.00070465169721295462,
    1.7464279966628027e-08]
TIME = [0] * 4  # Dummy values.
X = [
    np.array([1.]),
    np.array([-0.29187513]),
    np.array([-0.40719141]),
    np.array([-0.40777669])]
TRUE_HISTORY = {'func': FUNC,
                'grad_norm': GRAD_NORM,
                'time': TIME,
                'x': X}
# Constant step size.
opt = Newton(
    oracle, x0,
    tolerance=1e-10,
    line_search_options={
        'method': 'Constant',
        'c': 1.0}
)
opt.run(10)

(array([-0.40777669]),
 'success',
 defaultdict(list,
             {'func': [array([2.14872127]),
               array([0.9068072]),
               array([0.89869455]),
               array([0.89869434])],
              'time': [0.0, 0.000999, 0.000999, 0.000999],
              'grad_norm': [1.824360635350064,
               0.1402306959448993,
               0.0007046516972129546,
               1.7464279966628027e-08],
              'x_star': array([-0.40777669])}))

In [6]:
opt.hist

defaultdict(list,
            {'func': [array([2.14872127]),
              array([0.9068072]),
              array([0.89869455]),
              array([0.89869434])],
             'time': [0.0, 0.000999, 0.000999, 0.000999],
             'grad_norm': [1.824360635350064,
              0.1402306959448993,
              0.0007046516972129546,
              1.7464279966628027e-08],
             'x_star': array([-0.40777669])})

In [5]:
A = np.array([[1, 0], [0, 2]])
b = np.array([1, 6])
oracle = QuadraticOracle(A, b)

f_star = -9.5
x0 = np.array([0, 0])

In [6]:
opt = LBFGS(oracle, x0, tolerance=1e-5)
opt.run(5)
x_min = opt.hist['x_star']
      
        # x_min, message, _ = LBFGS(self.oracle, self.x0, tolerance=1e-5)
      
f_min = oracle.func(x_min)

In [7]:
g_k_norm_sqr = norm(self.A.dot(x_min) - self.b, 2)**2
g_0_norm_sqr = norm(self.A.dot(self.x0) - self.b, 2)**2

NameError: name 'norm' is not defined

In [16]:
opt.hist

defaultdict(list,
            {'func': [0.0,
              -9.376712328767121,
              -9.470440278207613,
              -9.5,
              -9.5,
              nan],
             'time': [0.0, 0.01, 0.01, 0.011001, 0.013002, 0.015001],
             'grad_norm': [6.082762530298219,
              0.49995308468204547,
              0.24480461387879468,
              0.0,
              0.0,
              nan],
             'x': [array([0, 0]),
              array([0.50684932, 3.04109589]),
              array([0.75852622, 3.02012282]),
              array([1., 3.]),
              array([1., 3.]),
              array([nan, nan])],
             'x_star': array([nan, nan])})

In [4]:
class TestBFGS():
    # Define a simple quadratic function for testing
    A = np.array([[1, 0], [0, 2]], dtype=np.float64)
    b = np.array([1, 6], dtype=np.float64)
    oracle = QuadraticOracle(A, b)

    f_star = -9.5
    x0 = np.array([0, 0], dtype=np.float64)
    # For this func |nabla f(x)| < tol ensures |f(x) - f(x^*)| < tol^2

    def test_default(self):
        """Check if everything works correctly with default parameters."""
        opt = BFGS(self.oracle, self.x0)
        opt.run()

    def test_tolerance(self):
        """Check if argument `tolerance` is supported."""
        BFGS(self.oracle, self.x0, tolerance=1e-5)

    def test_max_iter(self):
        """Check if argument `max_iter` is supported."""
        opt = BFGS(self.oracle, self.x0)
        opt.run(max_iter=0)

    def test_line_search_options(self):
        """Check if argument `line_search_options` is supported."""
        BFGS(self.oracle, self.x0, line_search_options={'method': 'Wolfe', 'c1': 1e-4, 'c2': 0.9})

    def test_quality(self):
        opt = BFGS(self.oracle, self.x0, tolerance=1e-5)
        opt.run(5)
        x_min = opt.hist['x_star']
        # x_min, message, _ = LBFGS(self.oracle, self.x0, tolerance=1e-5)
        f_min = self.oracle.func(x_min)

        g_k_norm_sqr = norm(self.A.dot(x_min) - self.b, 2)**2
        g_0_norm_sqr = norm(self.A.dot(self.x0) - self.b, 2)**2
        self.assertLessEqual(g_k_norm_sqr, 1e-5 * g_0_norm_sqr)
        self.assertLessEqual(abs(f_min - self.f_star), 1e-5 * g_0_norm_sqr)

    def test_history(self):
        x0 = -np.array([1.3, 2.7])
        opt = BFGS(self.oracle, x0, line_search_options={'method': 'Constant', 'c': 1.0}, tolerance=1e-6)
        opt.run(10)
        x_min = opt.hist['x_star']
        func_steps = [25.635000000000005,
                      22.99,
                      -9.48707349065929,
                      -9.5]
        grad_norm_steps = [11.629703349613008,
                           11.4,
                           0.22738961577617722,
                           0.0]
        time_steps = [0.0] * 4  # Dummy values
        x_steps = [np.array([-1.3, -2.7]),
                   np.array([1.0, 8.7]),
                   np.array([1.0, 2.88630519]),
                   np.array([1., 3.])]
        true_history = dict(grad_norm=grad_norm_steps, time=time_steps, x=x_steps, func=func_steps)
        check_equal_histories(opt.hist, true_history)

In [8]:
A = np.array([[1, 0], [0, 2]], dtype=np.float64)
b = np.array([1, 6], dtype=np.float64)
oracle = QuadraticOracle(A, b)

f_star = -9.5
x0 = np.array([0, 0], dtype=np.float64)

In [9]:
x0 = -np.array([1.3, 2.7])
opt = BFGS(oracle, x0, line_search_options={'method': 'Constant', 'c': 1.0}, tolerance=1e-6)
opt.run(10)

(array([1., 3.]),
 'success',
 defaultdict(list,
             {'func': [25.635000000000005,
               22.989999999999988,
               -9.48707349070719,
               -9.499999999999998],
              'time': [0.0, 0.0, 0.001002, 0.001002],
              'grad_norm': [11.629703349613008,
               11.399999999999999,
               0.22738961535490176,
               1.716138342544582e-11],
              'x': [array([-1.3, -2.7]),
               array([1. , 8.7]),
               array([1.        , 2.88630519]),
               array([1., 3.])],
              'x_star': array([1., 3.])}))